In [1]:
!pip install orjson -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 1.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast, os, re, json, requests, orjson
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''

---

In [3]:
txts = glob(data_folder+'data/raw_txt/*')
len(txts)

1603

In [4]:
titles = ['1. NAME OF THE MEDICINAL PRODUCT','2. QUALITATIVE AND QUANTITATIVE COMPOSITION', '3. PHARMACEUTICAL FORM', '4. CLINICAL PARTICULARS',
          '5. PHARMACOLOGICAL PROPERTIES', '6. PHARMACEUTICAL PARTICULARS', '7. MARKETING AUTHORISATION HOLDER', '8. MARKETING AUTHORISATION NUMBER',
          '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION','10. DATE OF REVISION OF THE TEXT']

In [ ]:
text_extracted = []
for txt in tqdm(txts):
  drug = txt.split('/')[-1].replace('.txt','')
  with open(txt, 'r+') as fi:
    txt = fi.read()
    txt = txt.split('ANNEX II')[0] #remove unneeded section
  for title in titles:
    if title != titles[-1]:
      next_title = titles[titles.index(title)+1]
      content = txt[txt.find(title):txt.find(next_title)]
      text_extracted.append([drug, title, content])
    else:
      content = txt[txt.find(title):]
      text_extracted.append([drug, title, content])

100%|██████████| 1603/1603 [01:07<00:00, 23.75it/s] 


In [ ]:
text_extracted_df = pd.DataFrame(text_extracted, columns=['drug', 'title', 'content'])
text_extracted_df.to_csv(data_folder+'data/text_extracted_raw.csv', index=False, escapechar = '\\')

In [ ]:
nosubsection = text_extracted_df[text_extracted_df['title'].isin(['4. CLINICAL PARTICULARS','5. PHARMACOLOGICAL PROPERTIES', '6. PHARMACEUTICAL PARTICULARS'])]

In [ ]:
#extract subsections
subsection_4 = ['4. CLINICAL PARTICULARS', '4.1 Therapeutic indications', '4.2 Posology and method of administration',
                '4.3 Contraindications', '4.4 Special warnings and precautions for use',
                '4.5 Interactions with other medicinal products and other forms of interaction',
                '4.6 Fertility, pregnancy, and lactation', '4.7 Effects on ability to drive and use machines',
                '4.8 Undesirable effects','4.9 Overdose']
subsection_5 = ['5. PHARMACOLOGICAL PROPERTIES','5.1 Pharmacodynamic properties',
                '5.2 Pharmacokinetic properties', '5.3 Preclinical safety data']
subsection_6 = ['6. PHARMACEUTICAL PARTICULARS','6.1 List of excipients','6.2 Incompatibilities',
                '6.3 Shelf life','6.4 Special precautions for storage',
                '6.5 Nature and contents of container','6.6 Special precautions for disposal and handling of product',]

In [ ]:
subsection_df = pd.DataFrame(columns=['drug', 'title', 'content'])
for subsection in [subsection_4, subsection_5, subsection_6]:
  main = subsection[0]
  sub_df = text_extracted_df[text_extracted_df['title'] == main]
  sub_list = []
  for i, row in tqdm(sub_df.iterrows()):
    txt = row['content']
    drug = row['drug']
    title = row['title']
    for title in subsection:
      if title != subsection[-1]:
        next_title = subsection[subsection.index(title)+1]
        content = txt[txt.find(title):txt.find(next_title)]
        sub_list.append([drug, title, content])
      else:
        content = txt[txt.find(title):]
        sub_list.append([drug, title, content])
  sub_df = pd.DataFrame(sub_list, columns=['drug', 'title', 'content'])
  subsection_df = pd.concat([subsection_df, sub_df])

1603it [00:00, 2347.48it/s]
1603it [00:00, 4413.27it/s]
1603it [00:00, 5164.84it/s]


In [ ]:
all_df = pd.concat([text_extracted_df, subsection_df])
all_df.to_csv(data_folder+'data/text_extracted_raw_subsection.csv', index=False, escapechar = '\\')

In [ ]:
all_df = all_df.rename(columns = {'drug':'product_id', 'title':'section_id', 'content':'section_content'})
all_df['section_title'] = all_df['section_id']
all_df.to_csv(data_folder+'data/drug_content.csv', index=False, escapechar = '\\')

----

In [5]:
all_df = pd.read_csv(data_folder+'data/drug_content.csv')
print(all_df.shape[0], all_df.product_id.nunique())
all_df.head(1)

49693 1603


,product_id,section_id,section_content,section_title
0,extavia,1. NAME OF THE MEDICINAL PRODUCT,1. NAME OF THE MEDICINAL PRODUCT \n \nExtavia...,1. NAME OF THE MEDICINAL PRODUCT


In [ ]:
#move this to the final folder
!cp $data_folder'data/drug_content.csv' $data_folder'data/final_EU/drug_content_all.csv'

----

In [9]:
df = pd.read_excel(data_folder+'data/Medicines_output_european_public_assessment_reports.xlsx', skiprows=8)
df = df[df.Category == 'Human']
df['Medicine name'] = df['Medicine name'].apply(lambda x: x.lower())
print(df.shape[0])
df.head(1)

1706


,Category,Medicine name,Therapeutic area,International non-proprietary name (INN) / common name,Active substance,Product number,Patient safety,Authorisation status,ATC code,Additional monitoring,...,Vet pharmacotherapeutic group,Date of opinion,Decision date,Revision number,Condition / indication,Species,ATCvet code,First published,Revision date,URL
0,Human,alecensa,"Carcinoma, Non-Small-Cell Lung",alectinib,alectinib hydrochloride,EMEA/H/C/004164,no,Authorised,L01ED03,yes,...,NaN,2016-12-15 01:00:00,2023-03-15 01:00:00,14.0,Alecensa as monotherapy is indicated for the f...,NaN,NaN,2018-08-09 15:09:00,2023-03-29 15:57:00,https://www.ema.europa.eu/en/medicines/human/E...


In [10]:
drug_info = df[['Product number', 'Medicine name', 'Active substance', 'Human pharmacotherapeutic group', 'Revision date',
   'Marketing authorisation holder/company name']]
drug_info.columns = ['product_number', 'product_id', 'ingredients', 'about', 'last_updated', 'company_title']
drug_info['contact_items'] = None
drug_info.to_csv(data_folder+'data/drug_info.csv', index=False, escapechar = '\\')
drug_info.head(1)

<ipython-input-10-a45502acd53c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_info['contact_items'] = None


,product_number,product_id,ingredients,about,last_updated,company_title,contact_items
0,EMEA/H/C/004164,alecensa,alectinib hydrochloride,Antineoplastic agents,2023-03-29 15:57:00,Roche Registration GmbH,None


In [11]:
#move this to the final folder
!cp $data_folder'data/drug_info.csv' $data_folder'data/final_EU/drug_info_all.csv'

-----

In [13]:
drug_info_df = pd.read_csv(data_folder+'data/drug_info.csv')
drug_content_df = pd.read_csv(data_folder+'data/drug_content.csv')

In [12]:
!mkdir $data_folder'data/xml_data'

In [20]:
import xml.etree.ElementTree as ET
#format into individual drug label XML files
def drug_label_xml(drug_id, out_folder):
    drug_info = drug_info_df[drug_info_df.product_id == drug_id]
    drug_content = drug_content_df[drug_content_df.product_id == drug_id]

    root = ET.Element("drug_label")

    drug_i = ET.Element("drug_info")
    root.append(drug_i)

    for col in drug_info.columns:
      subelement = ET.SubElement(drug_i, col)
      if str(drug_info[col].iloc[0]) != 'nan':
        subelement.text = str(drug_info[col].iloc[0])
      else:
        subelement.text = ''

    drug_label = ET.Element("drug_label_info")
    root.append(drug_label)

    for i, row in drug_content.iterrows():
      subelement = ET.SubElement(drug_label, row['section_id'])
      subelement_title = ET.SubElement(subelement, 'title')
      if str(row['section_id']) != 'nan':
        subelement_title.text = row['section_id']
      else:
        subelement_title.text = ''
      subelement_content = ET.SubElement(subelement, 'content')
      if row['section_content'] != None and row['section_content'] != '':
        subelement_content.text = str((row['section_content']))
      else:
        subelement_content.text = ''

    tree = ET.ElementTree(root)

    with open (out_folder+'{}.xml'.format(str(drug_id).replace('/','-')), "wb") as f:
        tree.write(f, encoding = 'utf-8')

In [21]:
for drug_id in tqdm(drug_info_df.product_id.unique()):
  drug_label_xml(drug_id, data_folder+'data/xml_data/')

100%|██████████| 1695/1695 [00:36<00:00, 46.38it/s]


In [ ]:
#out_file = data_folder+'data/final_UK/drug_label_xml.tar.gz'
#xml_dir = data_folder+'xml_data'
!tar -czf $data_folder"data/final_EU/drug_label_xml.tar.gz" $data_folder"data/xml_data"

---